# Creating a map

There are lots of ways to create a map.

Today we will cover the basics using `matplotlib` and `seaborn`. 

First, we will extract the data just for Grenada.

In [2]:
# Example
import os
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

filename = "ppp_2020_1km_Aggregated.tif"
folder = os.path.join('..', 'data', 'raw', 'worldpop')
path_population = os.path.join(folder, filename)

filename = 'gadm36_{}.shp'.format(1)
path_in = os.path.join('..', 'data', 'processed', 'GRD', 'regions', filename) 
boundaries = gpd.read_file(path_in)

output = []
for idx, boundary in boundaries.iterrows():
        
    print("Working on {}".format(boundary['GID_1']))
    
    with rasterio.open(path_population) as src:

        affine = src.transform                       #here we load the affine function 
        array = src.read(1)                          #here we load our actual data from layer 1
        array[array <= 0] = 0                        #here we get rid of negative number 

        population = [i['sum'] for i in zonal_stats( #now we sum all cells within our zonal boundary 
            boundary['geometry'],   #<- providing our boundary
            array,                  #<- providing our .tif raster data 
            nodata=255,             #<- stating the value that cells with no data will have
            stats=['sum'],          #<- stating what statistics we want
            affine=affine)][0]      #<- providing the affine function to convert between coordinate systems

        output.append({
            'GID_1': boundary['GID_1'],
            'population': population
        })
        
output = pd.DataFrame(output)
path_out = os.path.join('..', 'data', 'processed', 'GRD', 'population.csv')  
output.to_csv(path_out)

Working on GRD.1_1
Working on GRD.2_1
Working on GRD.3_1
Working on GRD.4_1
Working on GRD.5_1
Working on GRD.6_1
Working on GRD.7_1


Now we can load in our `population.csv` file and begin plotting. 

In [28]:
# Example
import matplotlib.pyplot as plt
import seaborn as sns

#import our boundaries data
filename = 'gadm36_{}.shp'.format(1)
path_in = os.path.join('..', 'data', 'processed', 'GRD', 'regions', filename) 
boundaries = geopandas.read_file(path_in)

#import our population data
filename = 'population.csv'
path_in = os.path.join('..', 'data', 'processed', 'GRD', filename) 
data = pd.read_csv(path_in)

#merge our population data onto our boundaries 
boundaries = boundaries.merge(data, left_on='GID_1', right_on='GID_1')

#define value bins and then labels for each one
bins = [-1e6, 2000, 4000, 6000, 8000, 1e12]
labels = ['<2k','2-4k','4-6k','6-8k','>8k']

#create a new variable with our bin labels
boundaries['bin'] = pd.cut(
    boundaries['population'],
    bins=bins,
    labels=labels
)

#open a new seaborn figure
sns.set(font_scale=0.9)
fig, ax = plt.subplots(1, 1, figsize=(4.5, 4.5))

#now plot our data using pandas plot
base = boundaries.plot(column='bin', ax=ax, cmap='viridis', linewidth=0, #inferno_r
    legend=True, antialiased=False)

#allocate a plot title 
n = len(boundaries)
name = 'Population by Sub-Region for Grenada (n={})'.format(n)
fig.suptitle(name)

#specify where to write our .png file to
path = os.path.join('..', 'data', 'processed', 'GRD', 'fig.png')
fig.savefig(path)
plt.close(fig)


# Exercise 1.1

Can you better specify the bins and labels to really demonstrate the spatial differences in population?

In [30]:
# Example code here


# Exercise 1.2

Can you create a plot for Barbados?


In [29]:
# Example code here
